In [20]:
import pandas as pd
#data input
industrialDemandH2 = pd.read_csv('/Users/kanankhasmammadov/Desktop/Thesis - Electrolyzer market participation/flexABLE_w_electrolyzer/input/2016/industrial_demand.csv')
electrolyzers = pd.read_csv('/Users/kanankhasmammadov/Desktop/Thesis - Electrolyzer market participation/flexABLE_w_electrolyzer/input/2016/electrolyzers.csv')
industrialDemandH2['Timestamp'] = pd.date_range(start=f'1/1/2016', end=f'12/31/2016 23:45', freq='15T')            

print(industrialDemandH2.head())
electrolyzers.head()

   tick  Elec_x  Metal           Timestamp
0     0   180.2    0.0 2016-01-01 00:00:00
1     1   179.1    0.0 2016-01-01 00:15:00
2     2   178.0    0.0 2016-01-01 00:30:00
3     3     0.0    0.0 2016-01-01 00:45:00
4     4     0.0    0.0 2016-01-01 01:00:00


,name,technology,minLoad,installedCapacity,effElec,minDowntime,minRuntime,coldStartUpCost,maxAllowedColdStartups,standbyCons,comprCons,maxSOC,company
0,Elec_x,PEM,0.05,60,0.7,0.5,1,50,3000,0.0005,0.0012,22700,Refining
1,Metal,PEM,0.05,190,0.7,0.5,1,50,4000,0.0020,0.0012,5700,Metallurgy


### Electrolyzer capacity calculation

The electrolyzer capacity should be able to produce hydrogen at each step without having need for storage. Therefore the maximum hydrogen demand value is extracted on study year and respective electrolyzer power is calculated for it. Since data frequency is 15 minute, that should also be considered in calculation.

In [21]:
energyContentH2_LHV = 0.03333 #MWh/kg or lower heating value of H2
efficiency = electrolyzers['effElec']
# Calculate electrolyzer capacity
Elec_x_maxDemand = max(industrialDemandH2['Elec_x'])
Metal_maxDemand = max(industrialDemandH2['Metal'])
print(Elec_x_maxDemand, 'Elec_x_maxDemand')
print(Metal_maxDemand, 'Metal_maxDemand')

Elec_x_Capacity = round((Elec_x_maxDemand * energyContentH2_LHV / efficiency[0]/ 0.25), -1)
Metal_Capacity = round((Metal_maxDemand * energyContentH2_LHV / efficiency[1] / 0.25), -1)

print(Elec_x_Capacity, 'Elec_x_Capacity')
print(Metal_Capacity, 'Metal_Capacity')

329.8 Elec_x_maxDemand
999.4263172 Metal_maxDemand
60.0 Elec_x_Capacity
190.0 Metal_Capacity


### Storage capacity calculation
Storage capacity is calculated based on assumption of having 1 day of enough hydrogen storage to proceed with operations.

In [18]:
from statistics import mean
# Calculate the maxSOC for H2 storage
demandSum_Elec_x = [] #weekly or daily demand sum
demandSum_Metal = [] #weekly or daily demand sum

industrialDemandH2['Date'] = industrialDemandH2['Timestamp'].dt.date
unique_days = industrialDemandH2['Date'].unique()
for day in unique_days:
    dailyIntervalDemand = industrialDemandH2[industrialDemandH2['Date'] == day]
    daily_sum_Elec_x = sum(dailyIntervalDemand['Elec_x'])
    daily_sum_Metal = sum(dailyIntervalDemand['Metal'])
    
    demandSum_Elec_x.append(daily_sum_Elec_x) 
    demandSum_Metal.append(daily_sum_Metal)        

storageCap_Elec_x = round(mean(demandSum_Elec_x), -2)
storageCap_Metal = round(mean(demandSum_Metal), -2)

print(storageCap_Elec_x, 'storageCap_Elec_x')
print(storageCap_Metal, 'storageCap_Metal')


22700.0 storageCap_Elec_x
5700.0 storageCap_Metal
